### 학습용 데이터셋 수집

### 네이버 플레이스 음식점 ID 수집

In [8]:
import numpy as np
import pandas as pd
import re
import time
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [4]:
title_list = pd.DataFrame()
titles = []
ids = []

url = 'https://store.naver.com/restaurants/list?bounds=121.0849205%3B34.184039%3B135.1364342%3B40.9669969&query=%EB%A7%9B%EC%A7%91&sessionid=dErvWZ6KBTCFitPSlhnkhdDu&sortingOrder=reviewCount'

# driver = webdriver.Chrome(executable_path='E:/Big_Data_Study/python/chromedriver.exe') # MAC
driver = webdriver.Chrome(executable_path='../chromedriver') # Windows
driver.implicitly_wait(1)

driver.get(url)

for i in range(15):
    time.sleep(2)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    title = soup.select('a.name')
    
    for t in title:
        titles.append(str(t['title']))
        ids.append(int(t['href'].split('id=')[1].split('&query')[0]))
    
    driver.find_element_by_class_name('num.current + a').click()

title_list['상호명'] = titles
title_list['ID'] = ids
title_list.to_csv('./data/음식점_리스트_학습용.csv',index=False)

### 리뷰 수집

In [ ]:
review_df = pd.DataFrame()
review_list = []
point_list = []
df = pd.read_csv('./data/음식점_리스트_학습용.csv')

In [19]:
def review_tab0304():
    driver.page_source
    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[1]/a[3]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # tab03 리뷰
    if soup.select_one('span.total'): # 페이지체크
        total_page = int(soup.select_one('span.total').text)
        if total_page >100:
            total_page = 100
        for i in range(total_page):
            time.sleep(0.5)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            if soup.select('div.inner'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)

                reviews = soup.select('div.info_area > div.review_txt > div.inner')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))

                if driver.find_element_by_xpath('//*[@id="panel03"]/div/div[4]/a[2]'):
                    driver.find_element_by_xpath('//*[@id="panel03"]/div/div[4]/a[2]').click()
#                 if driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[4]/a[2]'):
#                     driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[4]/a2]').click()
            else:
                break
                    
    driver.get(url)
    
    time.sleep(1)
    
    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[1]/a[4]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # tab04 리뷰
    if soup.select_one('span.total'): # 페이지체크

        total_page = int(soup.select_one('span.total').text)
        if total_page >100:
            total_page = 100
        for i in range(total_page):
            time.sleep(0.5)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            if soup.select('div.review_txt'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)

                reviews = soup.select('div.info_area > div.review_txt')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))

                if driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]'):
                    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]').click()
            else:
                return

In [20]:
def review_tab03():
    driver.page_source
    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[1]/a[3]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    if soup.select_one('span.total'): # 페이지체크

        total_page = int(soup.select_one('span.total').text)
        if total_page > 100:
            total_page = 100
        for i in range(total_page):
            time.sleep(0.5)
            html = driver.page_source
            soup = bs(html, 'html.parser')

            if soup.select('div.review_txt'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)

                reviews = soup.select('div.info_area > div.review_txt')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))

                if driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]'):
                    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]').click()
            else:
                driver.refresh()

In [25]:
# driver = webdriver.Chrome(executable_path='E:/Big_Data_Study/python/chromedriver.exe') # Windows
driver = webdriver.Chrome(executable_path='../chromedriver') # MAC
driver.implicitly_wait(1)

for i in range(len(df['ID'])):
    # 리뷰 페이지 이동
    url = 'https://store.naver.com/restaurants/detail?entry=pll&id={}'.format(df['ID'][i])
    driver.get(url)
    
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # 예약자 -> 영수증
    try:
        tab03_br_check = soup.select_one('div.place_tab_area > a#tab03')['aria-label']
        tab04_br_check = soup.select_one('div.place_tab_area > a#tab04')['aria-label']
    except:
        pass        
    
    if tab03_br_check == '예약자 리뷰':
        if tab04_br_check != '블로그 리뷰':
            review_tab0304()

    elif tab03_br_check != '블로그 리뷰':
        review_tab03()
        
    else:
        continue

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]"}
  (Session info: chrome=81.0.4044.138)


In [270]:
len(review_list), len(point_list)
print(len(review_list),len(set(review_list)))

63706 54940


In [41]:
review_df['review'] = review_list
review_df['point'] = point_list
df = pd.read_csv('./data/음식점_리뷰_데이터셋_학습용.csv', index_col=None)
len(df)

63706

In [46]:
df = pd.read_csv('./data/음식점_리뷰_데이터셋_학습용.csv', index_col=None)
print(len(df))
df = df.dropna()
df = df.drop_duplicates()
print(len(df))
df.to_csv('./data/음식점_리뷰_데이터셋_학습용.csv', index=False, encoding='utf-8')

78504
67168


In [47]:
df = pd.read_csv('./data/음식점_리뷰_데이터셋_학습용.csv', index_col=None)

posneg = []
for i in range(len(df)):
    if len(df['review'][i]) < 10:
        df['review'][i] = None
    if df['point'][i] >= 3.5:
        posneg.append(1)
    elif df['point'][i] < 2.5:
        posneg.append(0)
    else:
        posneg.append(2)

df['posneg'] = posneg

#송도맛집
포장도됨!
맛있게잘먹고왔어요
👍
만족했습니다
파스타 맛있어요!
양이 적은편
맛있었어요
맛있었어용,,!!
맛있었어유
만족합니다 
맛있어요~~
한식이 맛있어요
최고입니다.
맛잇어요 만족
가성비가 좋아요
아주좋습니다
마음에 듭니다
가성비가 좋네요
맛나요~~^^
가성비갑입니다
네 잘 먹었습니다
맛있네요 ㅎㅎ 
좋았습니다
맛잇게잘먹엇어용
매우만족했어요 
맛있어요~!
종류진짜 다양
맛잇엇어요~
너무 맛있어요
가성비 부페
맛있었습니다
맛있고 좋습니다.
마시따요오옹
저번이랑다르네여
맛있고 청결합니다
좋았슺니다
오늘도굿!!
주차빼고 만족
잘 먹고가요
잘머것습니다 
언제나만족
그냥 그랬ㅇ음..
매우만족했어요!
너무 좋았어요!
맛있는거 잘먹고 
좋았아요~
친절하고 좋습니다
후회없는 뷔페💖💖
한적하니 좋네요
만족스럽습니다ㅔ
서비스도  좋아요
너무별로네요
잘먹었습니다
만족합니다
가격만큼 입니다.
가성비 괜찮은 곳
음식이 맛있어요
잘 먹었습니다
가성비 갑!!
가성비짱 
맛있어요오
잘  먹었어요 
잘 먹었어요
무난했어요 
맛있었습니다
괜찮았어요
가성비 괜찬았어요
만족스러웠어요ㅋㅋ
만족스러웠어요
아주 좋았어요 
감사합니다
만족합니다!
좋아요     
좋았어요!
ㅇㄷㅂㄷㄱㄷㄱ
좋았어요.
만족합니다 
항상 만족합니다.
배가 아주 불러요
별로...
가성만 최고에요
맛있었습니다!
맛잇어용~!
좋았습니다
ㄱ만족만족
good !!
음식맛이 좋아요~
너무 맛있었어요
맛있었어요!
만족입니다. 
만족합니다.
그냥 괜찮았어요.
최고였어요 :)
괜찮아요
맛있는 쌈밥 
좋아요
맛있어요
맛있어요
좋아요 
좋아요
좋어요
조으네요
맛있어요
밑반찬도 맛있네요
맛있어요!
점점 아쉽네요
ㅇㅇ
맛나요
맛있습니다
굿
맛잇고 좋아요~~
완전맛집
맛있어용
맛있는 맛집
넘맛있고 친절해요
맛있어요
ㅅ
맛있어요~
굿굿
ㅎㅎㅎㅎ
귯귯
날파리나옴
수요미식회 맛집
굿
^^
굿
맛있었어요.
굿굿굿
가성비갑
조아요
좋아요
맛있엉슈
경치가 좋아요
기본합니다.
말이 필요없음
잘먹엇습니다
잘먹었습니다 
화이팅입니다
맛이 좋아요
추천합니

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



강추강추강
리우리우리우~ 
맛잇더영ㅇㅇㅇ
잘먹엇습니다.
진짜 맛잇어요
깔끔하고 맛나요~
배부른 샐러드에요
싸고 맛있어요
가성비 짱입니다
맛있어요최고입니다
강력추천함
진짜 맛있어요!
스프넘맛나용!
완전 만족합니다
잘먹었슴다
가성비 갑
매무만족합니다
가성비 좋았어요
가성비 짱이에요
오늘도 좋은하루
잘먹었스빈다.
맛있어요 ~~
항상 맛있어요
신선하고 밋나요~
만족스럽습니당
최고의 가성도시락
든든해요!!!
매우만족합니다
맛있게잘먹었습니다
가성비 굿
Good!!!
잘먹었습니다!!!
맛있었어요~!
잘먹었으여
만족쓰으으으
맛있어요 ㅎㅎ
항상 만족합니다
최고입니다!!!
진짜 맛나용
다이어트으
맛있어요 
맛잇어요요
신상메뉴 맛나요
가성비 👍🏻
잘먹엇습니다 
가성비 좋습니다
가성비 좋습니다.
맛있어요 최고
맛있어요 가성비굳
좋아요 감사합니다
스프넘맛있어요!!
잘 먹엇습니당~~
가성비 끝판대장
넘맛잇어요!!
가성비쵝오여
굿굿 싸고 맛있엉
잘먹었습니다아~
촤고에요 최고요
잚거었씁니다.
가성비좋아여
푸드코트내 맛집
맛있는 한식집
맛있음.
점심먹기 좋음.
굿굿굿굿굿이에요
무난햇던거 같아요
종류도많고 맛나용
맛있었어요!!
맛있습니당
맛났어요~~^^
잘먹었습니다 
음식은 맛있네요
굿 좋아요~ㅎㅎ
만족요~~~~
SOSO SOSO
맛있게 먹었습니다
맛있었어요~~
갈 만  합니다.
맛있게 먹었어요~
네 좋았습니다
맛있게잘먹었습니다
좋습니다.
맛있었어요
그저 그랬어요
그저 그저 그저
잘 먹었습니다!
안녕하세요 만족
좋았어요. 
연어 맛있음
매우 친절함
만족합니다..
보통이었습니다..
항상 맛잇음
먹을만하네요
맛있게잘먹었어요!
괜찮네요 ..
맛있고좋았어요
좋고
.
맛있아요
 
대박 좋아요 
잘이용했습니다
넘 마식
맛있어요.
맛있게잘먹었어요
매운만족입니다
잘 먹고왓습니다
맛있습니더.
만족했습니다~~
맛있었어요 ㅎ
넘좋았어요
잘무씀니다
맛있어요~~~~
맛있었어요^^
아주 친절해요
좋았어요ㅎㅎㅎㅎ
좋았습니다.
좋아요 맛있어요😀
굿굿굿!!!
대체로 만족합니다
언제가도맛있습니다
매우 좋아요!
친절하고 맛

잘먹었어요^^
괜찮았어요..
별로입니다
매우만족해용!!
맛있었어요 ㅎㅎ
만족 했습니다
속초갔으니깐~
너무맛있었어요
친절하고 좋아요
할인 받지 못함
짱맛있있었어요
완존히 맛있었어요
매우 맛있었어요
맛있네요~~
괜찮네요...
끄덕 끄덕
대게맛있네여
다녀왔어요
낫베드. 그럭
그냥 그런...
최고였어요ㅎㅎ
맛났어요~~!!
ㅇㅇㅇㅇㅇ
정말맛있어요
만족하였습니다
맛 괜창ㅎ았어요~
먹을만했습니다
맛있어요~~~~
친절합니다 맛나요
맛나요!!!!!
맛있음 친절함
친절하세요
별로였어요
맛있엇습니다
맛나고 친절
맛있어요!
잘먹었었습니다
맛있음요
물밀
😊
jmt
달아요
아 정말 맛있어요
역시 굿~~~
 맛있습니다.  
맛있어요ㅎㅎㅎㅎ
쵝오입니다~♡
최고
비빔이더맛있음
마시쪙
굿
맛집인정
부산밀면맛집
여전히 맛있어요
맛없어요 ㅜ
진짜 꿀맛 ㅠㅠ
진짜맛있어요
.
맛잇는데 비싸요
그냥 저냥
좋아요~~~+
좋았어요ㅎㅎ
잘 이용했어요.
만족했습니다~
매우 만족했어요~
좋았어요^^
맛있었어요~~!
직원들이 친절했요
변했어요. 
자리가 좁아요ㅠ
친절하고 편해요 
만족합니다.
친절하세요~
아주좋아요
친절하고 좋았어요
음식은 맛있었어요
맛있어요옹
괜찮았어요~
좋아요ㅎㅎ
맛있어요 친철해요
만족 합니다~^^
너무조아요
만족합니다!!
언제나 좋아요!
아주좋음역시 앗백
서비스가 좋았어요
애들이친절해
좋습니다!
서비스가 별로
고ㅑㄴ찮았숩니더~
맛있고좋아요~
나쁘지않았어요
ㅎㅎㅎㅎㅎㅎ
만족했어요
친절했습니다.
맛있고 친절해요
만족 합니다
좋았어욥~
별로에요~
만족합니다 :)
좋았네요 굿
좋았어요!! 굿굿
친절했어요
만족합니다~~
친절하세요~ㅎ
보통입니다
바로이용가능
서비스좋아요
맛있고 친절해요~
최고예요 !!!
너무좋아요
좋아요 좋습니다
좋아요좋아요좋아요
잘먹엇어요
맛있고 좋았어요!
믿고 먹는 아웃백
역시네요
맛나당
#아웃백 #대전
역시 아웃백!
맛있는 아웃백
맛있어용ㅋ
토마호크짱!!
포장도 좋아요 
만족
그럭저럭 가족외식
너무 추웠어요
#아웃백 #좋아요
친절해요 :)
너무 느끼했어요
쏘쏘
친절해요
좋음
좋

진짜 맛있어요!!
개꿀 맛집이요
맛있어요!ㅁ
양많고 맛나요!
아주좋았습니다.
가성비갑 
맛이 좋아요~ 
맛있습니다^^
최고의 맛~♡
만족하고 가요
엄청 맛있어요!!
아주 잘먹었습니다
맛있어요~ㅎㅎ
잘먹었어요 ~~
좋아요👍
좋았습니다!!! 
잘먹었습니다 :)
맛있습니다 ㅎㅎ
잘 먹었습니다 
잘 먹었습니다~!
잘먹어습니다～～
맛있었어요!!!
우삼겹 맛있어요
너무 맛나요~~~
매우만족했습니다!
맛잇어쏘요
잘먹어습니다~~
맛있어요♡ 
#교대맛집
제육 맛있네요~~
굳국
도야지
맛이쑴
정말 맛있어요 ㅎ
정말정말 맛있어욤
굿조아
아이스크림맛집
또 방문할게요
서버분 친절했음
맛있었어요~
맛있었습니다;)
맛있고좋아요!
ㅊ최고에요
좋아요ㅎㅎㅎ
맛있었습니다🙏🙏
역시 맛있어용
항상 친절합니다 
좋아요ㅎ짱입니다
늘 맛있어요
커플세트 맛잇어용
서비스.보통
맛잇었어용
움식두맛있구 짱짱
좋아요 추천합니다
아주 좋앗어요
굉장히 좋았어요~
역시 아웃백 했다
항상 친절합니다!
나쁘지않아요!!
만족입니다
맛있게 먹고 가용
맛잇져용🐽🐷🐖
친절햇어요
매번 좋아요^^
좋아요 맛잇고~~
괜찮았습니다^&
친절하고 깨끗해요
최고입니다~~
잘먹었습니다ㄴ
굿굿 맛잇어요.
감사합니당호호
잘먹고 왔어요
맛있어요ㅋㅋ
맛있네요 ㅋ
친절하고 좋아요!
맛있었어여
파스타가 별로에요
흠 그냥 보통이요
좋았어요쵝오
맛있고서비스좋네요
만족스럽네요
맛잇게잘먹앗어여
언제나 만족ㅎ
여러모로 좋아요
맛있게 먹었어요
괜찮아유ㅋ
아주 맛있어용
친절하네요
음식도맛있고좋아요
언제나 만족!!
맛잇었어요^^
맨날 맛나요
좋아여 구굿
아주좋네요
좋으아용ㅋㅋ
맛있어용~.~
잘알려주셨어요
친절하고맛있었어요
친절하십니다.
언제나 대만족!!
너무맛있어여!
너무 맛있었어용~
좋아요!!!!
좃아요~^^
좋았어요 좋아
써버 응대 별로
맛있고 좋았어요
맛있네요정말
친절하고좋았습니다
좋아요~ 좋아요~
맛나게 잘먹었어요
그냥그렇습니다
맛있게 먹었네여
좋네요~~
식사 잘 했어요
맛있었습니다 ㅎㅎ
추천드립니다
항상 맛있어요
Good!!
좋아요ㅎ 추천추천
언제나 만족!

푸짐하고 좋았어요
매우 매우좋아요
양 많고 맛있어요
맛있었어요 ^^
맛있었어요~~~~
굿 맛잇스
괜찮고 좋았어요
단골이에요
항상 훌륭합니다.
가성비좋아여^^
맛있어요~~^^
언제나맛족
굿굿굿궁궁
맛있엏엉ᆢㅡㅎ
좋았습니다.. 
분위기 좋네요
좋앗어요 ㅎㅎ
예약은 필수죠 ㅎ
친절해서 좋았어요
잘먹었습니다..
맛있어요! ㅎㅎ
친절하고맛있어요
넘 친절하세요
좋아요요오오오오오
잘 먹었어요~^^
맛나게 잘먹었어요
늘 맛있어요ㅎㅎ
노오토조ㅗ노콭
맛있어서 자주가요
너무 친절해용ㅎㅎ
믿고먹는 아웃백❤
..., .
진짜 좋았어요ㅎㅎ
좋앗어요감사합니다
괜찮아요 괜찮
만족해요~!
좋아요 맛있음
역시짱입니다
만족입니다.
굿 굿굿굿굿
좋아요ㅎㅎ굿
맛있어요 굿^^
만족합니다^^=
맛있어요최고
괜찮았어요 ㅎㅎ
너무 좋음 :D
서비스는 최고!
조아용 맛있구 짱
마싯엇어요
ㅋ좋았다많이
좋은하루였습니다
맛있네요. ^^
정말좋았어료
사람이너무많아요
굿ㅎㅎ맛있음
좋아요ㅋㅋㅋ
너무잘먹고왔어요
너무 친절하세요!
맛있습니다.
잘먹고왓어요ㅎㅎ
굿..펜타포트
좋습니다ㅎ
역시좋아요
기다림없이 입장
잘먹고왔습니다.
good!!
맛나게 먹고왔어요
예약하니 편하네요
맛있게 먹었어요ㅔ
앙기모링앙기모링
goooood
맛있어용 ㅎㅎㅋ
좋아요 ㅎㅎ
맛있어요 ㅎㅇㅎ
항상잘애용합니다
gooood
GOOOOOD
GOOOOOOOD
좋았네여~
gooooood
굿 좋앙ᆢㅇㅎㅎㅎ
goooooood
좋아 좋아요
맛나 
친저레
직원들 친절함
좋아요 맛나요
맛나유
맛있어영
굿굿 맛있어요.
할인찬스
 잘먹었어요
대체로 만족 
맛잇고좋아요
맛있고 좋습니다
맛나요~~^^
맛나요~~^^
맛있게 먹었습니다
대게가 없어요ㅠㅠ
나쁘지않음
너무좋아요 
참 맛나요.
항상 만족합니다
굳 너무좋았어요
항상 맛납니다~ 
맛있음 괜찮아유
만족하였습니다 ~
맛있게먹었어요..
그냥 평범., ,
맛있게먹고왔어요~
맛나요 ~
잘먹고왔어요~^^
맛있습니다~^^
잘 먹었어요!
잘 먹었어요 ^^
가을특가 최고^^
만족합니다..최고
분위기는 좋네요
맛있게잘먹었어요
아주 좋네

58124

In [48]:
df = df.dropna()
df.to_csv('./data/음식점_리뷰_데이터셋_학습용_전처리.csv', index=False, encoding='utf-8')

In [49]:
df = pd.read_csv('./data/음식점_리뷰_데이터셋_학습용_전처리.csv', index_col=None)
df

,review,point,posneg
0,맛있고 괜찮았다 냠냠,5.0,1
1,루꼴라 샐러드가 너무 맛있었습니다 !,3.5,1
2,맛도좋고 직원분들도 친절해요!,5.0,1
3,예상보다 좁았던 것 빼곤 음식은 긋긋이었어요,4.0,1
4,맛있어요 추천합니다,5.0,1
...,...,...,...
62168,항상 깔끔한 분위기에 맛도 좋아요,5.0,1
62169,까르보나라 너무 느끼함;;,2.0,0
62170,사람이 항상많은 곳 아이스아메리카노에서 약간의 산미가 느껴짐,3.0,2
62171,커피도 맛있어서 종종 들르는 곳,5.0,1


In [51]:
df_pos = df[df['posneg']==1]
df_neg = df[df['posneg']==0]
df_neu = df[df['posneg']==2]
display(df_pos,df_neg,df_neu)

,review,point,posneg
0,맛있고 괜찮았다 냠냠,5.0,1
1,루꼴라 샐러드가 너무 맛있었습니다 !,3.5,1
2,맛도좋고 직원분들도 친절해요!,5.0,1
3,예상보다 좁았던 것 빼곤 음식은 긋긋이었어요,4.0,1
4,맛있어요 추천합니다,5.0,1
...,...,...,...
62164,"호주의 유명한 카페라고 하던데, 나와는 좀.... 빨간 봉글레에 충격-.-);",4.0,1
62166,맛있는편입니다. 좋아요,4.0,1
62168,항상 깔끔한 분위기에 맛도 좋아요,5.0,1
62171,커피도 맛있어서 종종 들르는 곳,5.0,1


,review,point,posneg
7,가격에 비해 별루.. 스테이크는 맛있어요,2.0,0
173,예전에 갔을때 보다 음식도 별로고 음식 종류도 얼마 없었습니다. 그래서 할인률이 높...,1.5,0
193,할머니 생신으로 다녀왔습니다. 한번 더 오고 싶은 마음은 안들었습니다.,2.0,0
194,"가족 전체 할머니 생신으로 다녀왔지만, 또 예약해서 와야겠다는 생각은 안들었습니다.",2.0,0
199,코로나로 흉흉한 이시기에 어제 아홉명이 예약을하고 여덟명은방문을했고 한명은 어제갑자...,2.0,0
...,...,...,...
62156,음식의 맛이 난해합니다... 퀴노아 샐러드는 내용물끼리 맛이 어우러지지 못하며 소스...,1.0,0
62157,최악이에요. 그러기도 쉽지 않은데 샐러드는 조합도 안맞고.. 살다살다 치킨버거에 치...,1.0,0
62158,크리스마스즘 외국서온 친구와생일을 맞아 총 4명이 방문했습니다.건강이슈로 식단관리필...,1.0,0
62165,1. 까르보나라(21천원) - 느끼함. 가격대비 양이 너무 적음. 떫은맛이 남 2....,1.0,0


,review,point,posneg
8,파스타가 맛잇고 스톤스테이크는 별로네요 직접 구워먹어야 하네요 귀찮고 맛도 별로...,3.0,2
106,생각보다 맛이 아쉬웠습니다!!,3.0,2
164,조미료를 사용하지 않아서 싱거울 수 있다고 했는데 수프부터 짬,3.0,2
166,테라스 좌석이 있어서 그런지 문을 열어놔서 너무 더움 ㅜㅜ,3.0,2
167,5/21(목) 저녁(임직원 회식) 이용 후기 1. 좋았던점 1)사람이 많지 않아 번...,3.0,2
...,...,...,...
62147,생각보다 별로 가격대비 별로,3.0,2
62160,가격에 비해 맛이 그닥..까르보나라 넘 짜요; 피클같은것도 없고...음료랑 분위기는...,3.0,2
62163,맛은 별로에요 핫케이크 기대했는데 폭신하지도 않고 뭔가 퍽퍽한느낌입니다 파스타는 퓨...,3.0,2
62167,"천장도 높고 분위기는 좋은데, 가격 대비 음식 양이 넘 적다고 느꼈습니다ㅠㅠ",3.0,2


### 검증용 데이터셋 수집

### 네이버 플레이스 음식점 ID 수집

In [3]:
import numpy as np
import pandas as pd
import re
import time
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [7]:
title_list = pd.DataFrame()
titles = []
ids = []

url = 'https://store.naver.com/restaurants/list?=null&bounds=126.5077903%3B33.1981719%3B127.5508048%3B33.6629073&page=1&query=%EC%A0%9C%EC%A3%BC%EB%8F%84%20%EB%A7%9B%EC%A7%91&sessionid=dErvWZ6KBTCFitPSlhnkhdDu&sortingOrder=reviewCount'

driver = webdriver.Chrome(executable_path='E:/Big_Data_Study/python/chromedriver.exe')
# driver = webdriver.Chrome(executable_path='../chromedriver')
driver.implicitly_wait(1)

driver.get(url)

for i in range(15):
    time.sleep(2)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    title = soup.select('a.name')
    
    for t in title:
        titles.append(str(t['title']))
        ids.append(int(t['href'].split('id=')[1].split('&query')[0]))
    
    driver.find_element_by_class_name('num.current + a').click()

In [11]:
title_list['상호명'] = titles
title_list['ID'] = ids
title_list.to_csv('./data/음식점_리스트_검증용.csv',index=False)

### 리뷰 수집

In [21]:
review_df = pd.DataFrame()
review_list = []
point_list = []
add_list = []
title_list = []

df = pd.read_csv('./data/음식점_리스트_검증용.csv')

In [27]:
def review_tab0203():
    driver.page_source
    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[1]/a[2]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # tab03 리뷰
    if soup.select_one('span.total'): # 페이지체크
        total_page = int(soup.select_one('span.total').text)
        if total_page >20:
            total_page = 20
        for i in range(total_page):
            time.sleep(0.5)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            if soup.select('div.inner'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)

                reviews = soup.select('div.info_area > div.review_txt > div.inner')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))

                try:
                    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[4]/a[2]').click()
                except: break
            else:
                break
                    
    driver.get(url)
    
    time.sleep(1)
    
    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[1]/a[3]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # tab04 리뷰
    if soup.select_one('span.total'): # 페이지체크

        total_page = int(soup.select_one('span.total').text)
        if total_page >20:
            total_page = 20
        for i in range(total_page):
            time.sleep(0.5)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            if soup.select('div.review_txt'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)

                reviews = soup.select('div.info_area > div.review_txt')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))
                try:
                    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]').click()
                except: break
            else:
                break

In [28]:
def review_tab02():
    driver.page_source
    driver.find_element_by_xpath('/html/body/div/div/div/div[3]/div[1]/a[2]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    if soup.select_one('span.total'): # 페이지체크

        total_page = int(soup.select_one('span.total').text)
        if total_page > 20:
            total_page = 20
        for i in range(total_page):
            time.sleep(0.5)
            html = driver.page_source
            soup = bs(html, 'html.parser')

            if soup.select('div.review_txt'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)

                reviews = soup.select('div.info_area > div.review_txt')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))

                try:
                    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]').click()
                except:
                    break
            else:
                break

In [29]:
def review_tab0304():
    driver.page_source
    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[1]/a[3]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    title = soup.select_one('strong.name')
    add = soup.select_one('span.addr')
    
    # tab03 리뷰
    if soup.select_one('span.total'): # 페이지체크
        total_page = int(soup.select_one('span.total').text)
        if total_page >20:
            total_page = 20
        for i in range(total_page):
            time.sleep(0.5)
            
            html = driver.page_source
            soup = bs(html, 'html.parser')
            if soup.select('div.inner'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)
                    title_list.append(title.text)
                    add_list.append(add.text)

                reviews = soup.select('div.info_area > div.review_txt > div.inner')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))

                try:
                    driver.find_element_by_xpath('//*[@id="panel03"]/div/div[4]/a[2]').click()
                except:
                    break
            else:
                break
                    
    driver.get(url)
    
    time.sleep(1)
    
    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[1]/a[4]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    title = soup.select_one('strong.name')
    add = soup.select_one('span.addr')
    
    # tab04 리뷰
    if soup.select_one('span.total'): # 페이지체크

        total_page = int(soup.select_one('span.total').text)
        if total_page >20:
            total_page = 20
        for i in range(total_page):
            time.sleep(0.5)
            
            html = driver.page_source
            soup = bs(html, 'html.parser')
            if soup.select('div.review_txt'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)
                    title_list.append(title.text)
                    add_list.append(add.text)

                reviews = soup.select('div.info_area > div.review_txt')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))

                try:
                    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]').click()
                except:
                    break
            else:
                break

In [32]:
def review_tab03():
    driver.page_source
    driver.find_element_by_xpath('/html/body/div/div/div/div[3]/div[1]/a[3]').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    title = soup.select_one('strong.name')
    add = soup.select_one('span.addr')
    
    if soup.select_one('span.total'): # 페이지체크

        total_page = int(soup.select_one('span.total').text)
        if total_page > 20:
            total_page = 20
        for i in range(total_page):
            time.sleep(0.5)
            
            html = driver.page_source
            soup = bs(html, 'html.parser')

            if soup.select('div.review_txt'): # 리뷰체크
                points = soup.select('div.info_area > div.star_area > span.score')
                for point in points:
                    point_list.append(point.text)
                    title_list.append(title.text)
                    add_list.append(add.text)

                reviews = soup.select('div.info_area > div.review_txt')
                for review in reviews:
                    review_list.append(' '.join(review.text.split('\n')))

                try:
                    driver.find_element_by_xpath('/html/body/div/div/div[2]/div[3]/div[3]/div/div[2]/a[2]').click()
                except:
                    break
            else:
                break

In [36]:
driver = webdriver.Chrome(executable_path='E:/Big_Data_Study/python/chromedriver.exe') # Windows
# driver = webdriver.Chrome(executable_path='../chromedriver') # MAC
driver.implicitly_wait(1)

for i in range(len(df['ID'])):
    url = 'https://store.naver.com/restaurants/detail?entry=pll&id={}'.format(df['ID'][i])
    driver.get(url)
    
    time.sleep(1)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # 예약자 -> 영수증
    try:
        tab03_br_check = soup.select_one('div.place_tab_area > a#tab03')['aria-label']
        tab04_br_check = soup.select_one('div.place_tab_area > a#tab04')['aria-label']
    except:
        continue
    
    if tab03_br_check == '예약자 리뷰':
        if tab04_br_check != '블로그 리뷰':
            review_tab0304()

    elif tab03_br_check != '블로그 리뷰':
        review_tab03()
        
    else:
        continue

KeyError: 465

In [39]:
review_df['title'] = title_list
review_df['address'] = add_list
review_df['review'] = review_list
review_df['point'] = point_list
df.to_csv('./data/음식점_리뷰_데이터셋_검증용.csv', index=False, encoding='utf-8')

In [10]:
df = pd.read_csv('./data/음식점_리뷰_데이터셋_검증용.csv', index_col=None)
print(len(df))
df = df.dropna()
df = df.drop_duplicates()
print(len(df))
df.to_csv('./data/음식점_리뷰_데이터셋_검증용_전처리.csv', index=False, encoding='utf-8')

56090
24615


In [12]:
df = pd.read_csv('./data/음식점_리뷰_데이터셋_검증용_전처리.csv', index_col=None)

posneg = []
for i in range(len(df)):
    if len(df['review'][i]) < 10:
        df['review'][i] = None
    if df['point'][i] >= 3.5:
        posneg.append(1)
    elif df['point'][i] < 2.5:
        posneg.append(0)
    else:
        posneg.append(2)

df['posneg'] = posneg

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
print(len(df))
df = df.dropna()
print(len(df))
df.to_csv('./data/음식점_리뷰_데이터셋_검증용_전처리.csv', index=False, encoding='utf-8')

24615
21626


In [14]:
df1 = pd.read_csv('./data/음식점_리뷰_데이터셋_학습용_전처리.csv', index_col=None)
df2 = pd.read_csv('./data/음식점_리뷰_데이터셋_검증용_전처리.csv', index_col=None)

In [61]:
from sklearn.utils import shuffle

df1_0 = df1[df1['posneg']==0]
df1_1 = df1[df1['posneg']==1]

df1 = pd.concat([df1_0,df1_1])
df1 = shuffle(df1)
df1.to_csv('./data/음식점_리뷰_데이터셋_학습용_전처리_2.csv', index=False, encoding='utf-8',sep='\t')

df2 = shuffle(df2)
df2.to_csv('./data/음식점_리뷰_데이터셋_검증용_전처리_2.csv', index=False, encoding='utf-8',sep='\t')